### The script 
- will prepare brand data which is used to recommend up to 5 recommended next category.
- browse data = 49 and trans for recent 3 years.
Note: for transaction data, we can avoid itm_item_status is void or cancel as select * 
From nmedwprd_db.pdwdm.all_sas_sales_and_open_orders_v
where itm_item_status not in ('V','CX')

it can be done as itm_item_status ='S'
   
**Last update**: Feb 28, 2021

In [1]:
import gc
import sys
import numpy as np
import pandas as pd
import boto3
import gc

client = boto3.client('s3') 
resource = boto3.resource('s3')
from datetime import date,datetime, timedelta
sys.path.append('/home/ec2-user/SageMaker/Category_propensity_recs/')
from Functions import common_header as h, tools as t, data_functions as d, ml_functions as m

#Code Parameters
brand = 'nm'
env = 'prod'
wbrand='NM'

In [2]:
# Date parameters
window = 49
duration = 30            # duration to retrieve categories that customers purcchased on
today = date.today()
idx = (today.weekday() + 1) % 7
last_sat = today - timedelta(7+idx-6)

run_1yr  = today - timedelta(days= 365)
run_3yr  = last_sat - timedelta(days=365*3)
recent_duration = (last_sat - h.timedelta(days= duration), last_sat, 'nm')
days_browse     = [last_sat - h.timedelta(days=window), last_sat, 'nm']
days_browse, recent_duration

([datetime.date(2021, 1, 16), datetime.date(2021, 3, 6), 'nm'],
 (datetime.date(2021, 2, 4), datetime.date(2021, 3, 6), 'nm'))

## Retrieve browse data of 49 days

## Retrieved transaction data

In [3]:
# Transaction data created by Designer_division_class_historical_ranking script for customers who are active from 
# the last recent 3 years

def read_df_from_s3_parquet( save_dir_nameX, bucket_nameX):
    import pyarrow.parquet as pq
    import s3fs
    s3 = s3fs.S3FileSystem()
    
    df = pq.ParquetDataset("s3a://"+bucket_nameX+'/'+save_dir_nameX, filesystem=s3).read_pandas().to_pandas()
    return df


## Query trans data

In [4]:
## Using this function to filter out data depending oon category
def retrieve_trans(trans_df, categories):
    """  last 3 yearstransaction with transfomation implemented in Designer_division_class_historical_ranking 
    script
    """
    cols = ['cmd_id']+ categories
    return trans_df[cols]

In [5]:
def query_browse(category):
    """ Retrieve all browse data who perform any product search for the last 49 days
    """
    sql="""select curr_cmd_id cmd_id ,{0} from nmedwprd_db.mktsand.click_stream_data;""".format(category)
    df = t.SF_read_sql_with_duplicate(sql)
    df = df[df.cmd_id != '-1']
    return df

In [6]:
def concat_columns(df, cols_to_concat, new_col_name, sep=" "):
    """ concatenate multiple columns
    """
    tmp = df.copy(deep= True)
    col_list = tmp.columns.values
    tmp[new_col_name] = tmp[cols_to_concat[0]]
    for col in cols_to_concat[1:]:
        tmp[new_col_name] = tmp[new_col_name].astype(str) + sep + tmp[col].astype(str)
    cols = tmp.columns.values
    return tmp[[cols[0],cols[-1]] +list(cols[1:-1])]

In [7]:
def agg_category(browse_df, trans_df, category):
    """ Aggregate CLV features per customer including Monetary value, Frequency and Recency 
    """
    group = list(browse_df.columns)
    browse_df = browse_df.groupby(group).size().reset_index(name= 'Times_Browse')
    group = list(trans_df.columns)
    trans_df  = trans_df.groupby(group).size().reset_index(name='Total_transactions')
    # Merge profiles
    cust= pd.merge(browse_df, trans_df,  how='outer', on= group)
    del browse_df, trans_df
    
    cust.fillna(value=0, axis=1, inplace = True)
   
    #Multiplying Times_Browsed and Total_Transactions by weights to build customer category taste profile
    cust['Transactions_W'] = cust['Total_transactions'].apply(lambda x: x*0.8)
    cust['Browsed_W'] = cust['Times_Browse'].apply(lambda x: x*0.2)

    #Summing the two weighted columns to obtained implicit rank unscaled 
    cust['rank_implicit'] = cust['Transactions_W'] + cust['Browsed_W']
    cust = cust.dropna(subset=['rank_implicit']) #Dropping NAN values
    cust['rank_implicit'] = pd.to_numeric(cust['rank_implicit'], errors='coerce')
    
    #Bucketizing column 'rank_implicit' based on quantiles so that it ranges from 1-5
    cust['score'] = np.where(cust['rank_implicit'] < cust['rank_implicit'].quantile(0.25),1,
	np.where((cust['rank_implicit'] >= cust['rank_implicit'].quantile(0.25)) & (cust['rank_implicit'] < cust['rank_implicit'].quantile(0.5)),2,
	np.where((cust['rank_implicit'] >= cust['rank_implicit'].quantile(0.5)) & (cust['rank_implicit'] < cust['rank_implicit'].quantile(0.75)),3,
	np.where((cust['rank_implicit'] >= cust['rank_implicit'].quantile(0.75)) & (cust['rank_implicit'] < cust['rank_implicit'].quantile(0.85)),4,5))))
    cust.drop(['Times_Browse','Total_transactions','Transactions_W','Browsed_W','rank_implicit'], axis=1, inplace= True)
    #cust.columns = ['cmd_id']+[category]+['score']
    return cust

In [8]:
def save_df_to_s3_gz(df_to_save,  dir_nameX, file_nameX, bucket_nameX='nmg-analytics-ds-prod'):
    """
       dir_nameX  = 'ds/prod/brand_affinity/nm/output/'
       file_nameX = 'designer_division_class_rank.csv'
       save_df_to_s3_gz(df , dir_nameX, file_nameX )
    """
    import gzip
    from io import BytesIO, TextIOWrapper
    
    gz_buffer = BytesIO()

    with gzip.GzipFile(mode='w', fileobj=gz_buffer) as gz_file:
        df_to_save.to_csv(TextIOWrapper(gz_file, 'utf8'), index=False,header=True,sep=',')
   
    
    s3_object = resource.Object(bucket_nameX, dir_nameX +file_nameX +'.gz')
    s3_object.put(Body=gz_buffer.getvalue())    
 
    return True

## Designer 

In [1]:
category= 'designer'
browse_df = query_browse('designer as category, designer_id, designer')
browse_df.dropna(axis=0, how= 'any',inplace= True)
#browse_df.head()

NameError: name 'query_browse' is not defined

In [10]:
trans_df = read_df_from_s3_parquet( 'ds/prod/ClientConnect/CMD/data/nm/trans_3y_history.csv.gz','nmg-analytics-ds-prod')
trans_df = retrieve_trans(trans_df, ['designer','designer_id', 'designer'])
trans_df.columns = ['cmd_id','category','designer_id', 'designer']
# trans_df.head()

,cmd_id,category,designer_id,designer
0,17EtLPS8,Parker,14069,Parker
1,uaAJGq0,Johnny Was,12751,Johnny Was
2,qVD9A2t,Lafayette 148 New York,12797,Lafayette 148 New York
3,ryYPgRF,Jay Godfrey,12645,Jay Godfrey
4,ytUWIo1,Bobbi Brown,10781,Bobbi Brown


In [11]:
trans_df = agg_category(browse_df, trans_df,'category')


In [ ]:
dir_name = 'ds/prod/ClientConnect/CMD/data/nm/'
save_df_to_s3_gz(trans_df, dir_name, 'cust_designer_rank.csv', bucket_nameX='nmg-analytics-ds-prod')
